In [1]:
import numpy as np 
import random
import statistics as st

dictionary = set()
maxWordLength = 0

def load_data(file):
    global maxWordLength
    res = []
    count = 0
    with open(file) as f:
        for line in f:
            temp = [x.lower() for x in line.split(' ')]
            res.append(temp)
            count += 1
            if count > 1000:
                break
    count = 0
    with open(file) as f:
        for line in f:
            temp = [x.lower() for x in line.split(' ')]
            for el in temp:
                maxWordLength = max(len(el), maxWordLength)
                dictionary.add(el)
            count += 1
            if count > 50000:
                break
    return res

tokens = load_data('polish_corpora.txt')
    

    

In [2]:

def editDistance(s1, s2):
    m, n  = len(s1), len(s2)
    dp = np.zeros((n+1)*(m+1)).reshape(m+1, n+1)
    for i in range(m+1):
        for j in range(n+1):
            if i == 0:
                dp[i][j] = j
            elif j == 0:
                dp[i][j] = i
            elif s1[i-1] == s2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = 1 + min(dp[i][j-1], dp[i-1][j], dp[i-1][j-1]);
    return dp[m][n]



In [3]:
def maxSquareLength(text):
    dp = [0]*(len(text)+3)
    words = [0]*(len(text)+3)
    for i in range(len(text)+1):
        if i < maxWordLength:
            #print("sprawdzam:",text[0:i+1], i)
            if text[0:i+1] in dictionary:
                #print("weszlo")
                dp[i] = (i+1)**2
                words[i] = i+1
        for j in range(i, max(-1, i-maxWordLength), -1):
            # print(text[j:i+1], j, i+1)
            if text[j: i + 1] in dictionary and dp[j - 1] != 0:
                if dp[j-1] + (i-j+1)**2 > dp[i]:
                    words[i] = i-j+1
                dp[i] = max(dp[i], dp[j-1]+(i-j+1)**2)

    i = len(text)-1

    res_temp = []
    while i >= 0:
        #print(text[i-words[i]+1:i+1])
        if words[i] == 0:
            res_temp.append(text[i:i+1])
            i -= 1
        else:
            res_temp.append(text[i-words[i]+1:i+1])
            i -= words[i]

    res_temp.reverse()
    
    return res_temp

In [4]:

def MaxMatch(sentence):
    res = []
    i = maxWordLength
    while i > 0:
        if sentence[:i] in dictionary or len(sentence[:i]) == 1:
            res.append(sentence[:i])
            sentence = sentence[i:]
            i = maxWordLength
        else:
            i -= 1
    return res
    
maxMatchRes1 = []
maxSquareRes1 = []
maxMatchRes2 = []
maxSquareRes2 = []

for line in tokens:
    noSpace = "".join(line)
    maxMatchTokens = MaxMatch(noSpace)
    maxSquareLengthTokens = maxSquareLength(noSpace)
    # print('---------------------------------------------')
    # print(maxMatchTokens)
    # print(maxSquareLengthTokens)
    # print('---------------------------------------------')
    line = ' '.join(line)
    maxMatchTokens = ' '.join(maxMatchTokens)

    maxSquareLengthTokens = ' '.join(maxSquareLengthTokens)

    ed1 = editDistance(line, maxMatchTokens)
    ed2 = editDistance(line, maxSquareLengthTokens)

    maxMatchRes1.append(ed1 / len(noSpace))
    maxSquareRes1.append(ed2 / len(noSpace))
    maxMatchRes2.append(ed1)
    maxSquareRes2.append(ed2)



In [5]:
print('Wynik maxMatcha:', st.mean(maxMatchRes1),'->',st.stdev(maxMatchRes1), st.mean(maxMatchRes2), '->',st.stdev(maxMatchRes2))
print('Wynik maxSquare:', st.mean(maxSquareRes1),'->', st.stdev(maxSquareRes1), st.mean(maxSquareRes2),'->', st.stdev(maxSquareRes2))


Wynik maxMatcha: 0.07976379096619823 -> 0.06637538264012485 8.577422577422578 -> 8.224004483841783
Wynik maxSquare: 0.010585473080798748 -> 0.021216806771395088 1.01998001998002 -> 1.869117545688446
